In [ ]:
#import ipdb #for debugging add ipdb.set_trace()
import numpy as np
import sys
from scipy.sparse import csr_matrix
from scipy.linalg import eigh

#Linear Shape functions and its derivatives
def GetShapeFn(z, eta, k, order):
  N_ShapeFn = None
  if(order==1):
    N_ShapeFn = np.zeros( (1, 8) )
    N_ShapeFn[0][0] = (1-z)*(1-eta)*(1-k)/8
    N_ShapeFn[0][1] = (1+z)*(1-eta)*(1-k)/8
    N_ShapeFn[0][2] = (1+z)*(1+eta)*(1-k)/8
    N_ShapeFn[0][3] = (1-z)*(1+eta)*(1-k)/8

    N_ShapeFn[0][4] = (1-z)*(1-eta)*(1+k)/8
    N_ShapeFn[0][5] = (1+z)*(1-eta)*(1+k)/8
    N_ShapeFn[0][6] = (1+z)*(1+eta)*(1+k)/8
    N_ShapeFn[0][7] = (1-z)*(1+eta)*(1+k)/8
    
  if(order==2):
    N_ShapeFn = np.zeros( (1, 20) )
    N_ShapeFn[0][0]=-(1-z)*(1-eta)*(1-k)*(2+z+eta+k);
    N_ShapeFn[0][1]=-(1+z)*(1-eta)*(1-k)*(2-z+eta+k);
    N_ShapeFn[0][2]=-(1+z)*(1+eta)*(1-k)*(2-z-eta+k);
    N_ShapeFn[0][3]=-(1-z)*(1+eta)*(1-k)*(2+z-eta+k);
    N_ShapeFn[0][4]=-(1-z)*(1-eta)*(1+k)*(2+z+eta-k);
    N_ShapeFn[0][5]=-(1+z)*(1-eta)*(1+k)*(2-z+eta-k);
    N_ShapeFn[0][6]=-(1+z)*(1+eta)*(1+k)*(2-z-eta-k);
    N_ShapeFn[0][7]=-(1-z)*(1+eta)*(1+k)*(2+z-eta-k);
    N_ShapeFn[0][8]=(1-z*z)*(1-eta)*(1-k)*2;
    N_ShapeFn[0][9]=(1-eta*eta)*(1+z)*(1-k)*2;
    N_ShapeFn[0][10]=(1-z*z)*(1+eta)*(1-k)*2;
    N_ShapeFn[0][11]=2*(1-eta*eta)*(1-z)*(1-k);
    N_ShapeFn[0][12]=2*(1-z*z)*(1-eta)*(1+k);
    N_ShapeFn[0][13]=2*(1-eta*eta)*(1+z)*(1+k);
    N_ShapeFn[0][14]=2*(1-z*z)*(1+eta)*(1+k);
    N_ShapeFn[0][15]=2*(1-eta*eta)*(1-z)*(1+k);
    N_ShapeFn[0][16]=2*(1-k*k)*(1-z)*(1-eta);
    N_ShapeFn[0][17]=2*(1-k*k)*(1+z)*(1-eta);
    N_ShapeFn[0][18]=2*(1-k*k)*(1+z)*(1+eta);
    N_ShapeFn[0][19]=2*(1-k*k)*(1-z)*(1+eta);
    
  return N_ShapeFn

def GetShapeFnGradientwZ( z,  eta,  k, order):
  N_ShapeFnGradient =None
  if(order==1):
    N_ShapeFnGradient = np.zeros( (1, 8) )
    N_ShapeFnGradient[0][0] = -(1-eta)*(1-k)/8
    N_ShapeFnGradient[0][1] = (1-eta)*(1-k)/8
    N_ShapeFnGradient[0][2] = (1+eta)*(1-k)/8
    N_ShapeFnGradient[0][3] = -(1+eta)*(1-k)/8
    N_ShapeFnGradient[0][4] = -(1-eta)*(1+k)/8
    N_ShapeFnGradient[0][5] = (1-eta)*(1+k)/8
    N_ShapeFnGradient[0][6] = (1+eta)*(1+k)/8
    N_ShapeFnGradient[0][7] = -(1+eta)*(1+k)/8
  if(order==2):
     N_ShapeFnGradient = np.zeros( (1, 20) )
     N_ShapeFnGradient[0][0]= ((eta - 1)*(k - 1)*(eta + 2*z + k + 1))/8
     N_ShapeFnGradient[0][1]=-((eta - 1)*(k - 1)*(eta - 2*z + k + 1))/8
     N_ShapeFnGradient[0][2]=-((eta + 1)*(k - 1)*(eta + 2*z - k - 1))/8
     N_ShapeFnGradient[0][3]=-((eta + 1)*(k - 1)*(2*z - 2*z + k + 1))/8
     N_ShapeFnGradient[0][4]=-((eta - 1)*(k + 1)*(eta + 2*z - k + 1))/8
     N_ShapeFnGradient[0][5]=((eta - 1)* (k + 1)*(eta - 2*z - k + 1))/8
     N_ShapeFnGradient[0][6]=((eta + 1)* (k + 1)*(eta + 2*z + k - 1))/8
     N_ShapeFnGradient[0][7]=-((eta + 1)*(k + 1)*(eta - 2*z + k - 1))/8
     N_ShapeFnGradient[0][8]=-(z*(eta - 1)*(k - 1))/2
     N_ShapeFnGradient[0][9]=((pow(eta,2) - 1)*(k - 1))/4
     N_ShapeFnGradient[0][10]=(z*(eta + 1)*(k - 1))/2
     N_ShapeFnGradient[0][11]=-((2*pow(eta,2) - 2)*(k - 1))/8
     N_ShapeFnGradient[0][12]=(z*(eta - 1)*(k + 1))/2
     N_ShapeFnGradient[0][13]=-((2*pow(eta,2) - 2)*(k + 1))/8
     N_ShapeFnGradient[0][14]=-(z*(eta + 1)*(k + 1))/2
     N_ShapeFnGradient[0][15]=((2*pow(eta,2) - 2)*(k + 1))/8
     N_ShapeFnGradient[0][16]=-((2*pow(k,2) - 2)*(eta - 1))/8
     N_ShapeFnGradient[0][17]=((2*pow(k,2) - 2)*(eta - 1))/8
     N_ShapeFnGradient[0][18]=-((2*pow(k,2) - 2)*(eta + 1))/8 
     N_ShapeFnGradient[0][19]=((2*pow(k,2) - 2)*(eta + 1))/8
    
  return N_ShapeFnGradient

def GetShapeFnGradientwEta( z,  eta,  k,  order):
  N_ShapeFnGradient =None
  if(order==1):
    N_ShapeFnGradient = np.zeros( (1, 8) )
    N_ShapeFnGradient[0][0] = -(1-z)*(1-k)/8
    N_ShapeFnGradient[0][1] = -(1+z)*(1-k)/8
    N_ShapeFnGradient[0][2] = (1+z)*(1-k)/8
    N_ShapeFnGradient[0][3] = (1-z)*(1-k)/8
    N_ShapeFnGradient[0][4] = -(1-z)*(1+k)/8
    N_ShapeFnGradient[0][5] = -(1+z)*(1+k)/8
    N_ShapeFnGradient[0][6] = (1+z)*(1+k)/8
    N_ShapeFnGradient[0][7] = (1-z)*(1+k)/8
  if(order==2):
    N_ShapeFnGradient = np.zeros( (1, 20) )
    N_ShapeFnGradient[0][0]= ((z - 1)*(k - 1)*(2*eta + z + k + 1))/8
    N_ShapeFnGradient[0][1]=-((z + 1)*(k - 1)*(2*eta - z + k + 1))/8
    N_ShapeFnGradient[0][2]=-((z + 1)*(k - 1)*(2*eta + z - k - 1))/8
    N_ShapeFnGradient[0][3]=-((z - 1)*(k - 1)*(z - 2*eta + k + 1))/8
    N_ShapeFnGradient[0][4]=-((z - 1)*(k + 1)*(2*eta + z - k + 1))/8
    N_ShapeFnGradient[0][5]= ((z + 1)*(k + 1)*(2*eta - z - k + 1))/8
    N_ShapeFnGradient[0][6]= ((z + 1)*(k + 1)*(2*eta + z + k - 1))/8
    N_ShapeFnGradient[0][7]=-((z - 1)*(k + 1)*(2*eta - z + k - 1))/8
    N_ShapeFnGradient[0][8]=-((pow(z,2) - 1)*(k - 1))/4
    N_ShapeFnGradient[0][9]=(eta*(z + 1)*(k - 1))/2
    N_ShapeFnGradient[0][10]=((pow(z,2) - 1)*(k - 1))/4
    N_ShapeFnGradient[0][11]=-(eta*(z - 1)*(k - 1))/2
    N_ShapeFnGradient[0][12]=((2*pow(z,2) - 2)*(k + 1))/8
    N_ShapeFnGradient[0][13]=-(eta*(z + 1)*(k + 1))/2
    N_ShapeFnGradient[0][14]=-((2*pow(z,2) - 2)*(k + 1))/8
    N_ShapeFnGradient[0][15]=(eta*(z - 1)*(k + 1))/2
    N_ShapeFnGradient[0][16]=-((2*pow(k,2) - 2)*(z - 1))/8
    N_ShapeFnGradient[0][17]=((2*pow(k,2) - 2)*(z + 1))/8
    N_ShapeFnGradient[0][18]=-((2*pow(k,2) - 2)*(z + 1))/8
    N_ShapeFnGradient[0][19]=((2*pow(k,2) - 2)*(z - 1))/8
   
  return N_ShapeFnGradient

def GetShapeFnGradientwK( z,  eta,  k,  order):
  N_ShapeFnGradient =None
  if(order==1):
    N_ShapeFnGradient = np.zeros( (1, 8) )
    N_ShapeFnGradient[0][0] = -(1-z)*(1-eta)/8
    N_ShapeFnGradient[0][1] = -(1+z)*(1-eta)/8
    N_ShapeFnGradient[0][2] = -(1+z)*(1+eta)/8
    N_ShapeFnGradient[0][3] = -(1-z)*(1+eta)/8
    N_ShapeFnGradient[0][4] = (1-z)*(1-eta)/8
    N_ShapeFnGradient[0][5] = (1+z)*(1-eta)/8
    N_ShapeFnGradient[0][6] = (1+z)*(1+eta)/8
    N_ShapeFnGradient[0][7] = (1-z)*(1+eta)/8
  if(order==2):
    N_ShapeFnGradient = np.zeros( (1, 20) )
    N_ShapeFnGradient[0][0]= ((eta - 1)*(z - 1)*(eta + z + 2*k + 1))/8
    N_ShapeFnGradient[0][1]=-((eta - 1)*(z + 1)*(eta - z + 2*k + 1))/8
    N_ShapeFnGradient[0][2]=-((eta + 1)*(z + 1)*(eta + z - 2*k - 1))/8
    N_ShapeFnGradient[0][3]=-((eta + 1)*(z - 1)*(z - eta + 2*k + 1))/8
    N_ShapeFnGradient[0][4]=-((eta - 1)*(z - 1)*(eta + z - 2*k + 1))/8
    N_ShapeFnGradient[0][5]= ((eta - 1)*(z + 1)*(eta - z - 2*k + 1))/8
    N_ShapeFnGradient[0][6]= ((eta + 1)*(z + 1)*(eta + z + 2*k - 1))/8
    N_ShapeFnGradient[0][7]=-((eta + 1)*(z - 1)*(eta - z + 2*k - 1))/8
    N_ShapeFnGradient[0][8]=-((pow(z,2) - 1)*(eta - 1))/4
    N_ShapeFnGradient[0][9]=((pow(eta,2) - 1)*(z + 1))/4
    N_ShapeFnGradient[0][10]=((pow(z,2) - 1)*(eta + 1))/4
    N_ShapeFnGradient[0][11]=-((2*pow(eta,2) - 2)*(z - 1))/8
    N_ShapeFnGradient[0][12]=((2*pow(z,2) - 2)*(eta - 1))/8
    N_ShapeFnGradient[0][13]=-((2*pow(eta,2) - 2)*(z + 1))/8
    N_ShapeFnGradient[0][14]=-((2*pow(z,2) - 2)*(eta + 1))/8
    N_ShapeFnGradient[0][15]=((2*pow(eta,2) - 2)*(z - 1))/8
    N_ShapeFnGradient[0][16]=-(k*(eta - 1)*(z - 1))/2
    N_ShapeFnGradient[0][17]=(k*(eta - 1)*(z + 1))/2
    N_ShapeFnGradient[0][18]=-(k*(eta + 1)*(z + 1))/2
    N_ShapeFnGradient[0][19]=(k*(eta + 1)*(z - 1))/2
  return N_ShapeFnGradient

def GetElemArr( NXElems,  NYElems,  NZElems,  NNodesPerElem, order):
    Elem_LocaltoGlobalNodeArr = np.zeros( (NXElems*NYElems*NZElems,NNodesPerElem) )

    count = 0
    if(order==1):
      for i in range(NZElems):#in z direction
          for j in range(NYElems):#in y direction
              for k in range(NXElems):#in x direction
                  #first looping through all X dir elems then to Y dir and then to Z dir
                  nElemId = k+(j)*NXElems+(i)*NXElems*NYElems
                  #print('nElemId',nElemId)
                  nGNodeId1 =nElemId+(j)+(i)*(NXElems+NYElems+1)
                  nGNodeId2 =nElemId+(j+1)+(i)*(NXElems+NYElems+1)
                  nGNodeId3 =nElemId+(j+1)+NXElems+1+(i)*(NXElems+NYElems+1)
                  nGNodeId4 =nElemId+(j+1)+NXElems+(i)*(NXElems+NYElems+1)
                  nGNodeId5 =nElemId+(j)+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                  nGNodeId6 =nElemId+(j+1)+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                  nGNodeId7 =nElemId+(j+1)+NXElems+1+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                  nGNodeId8 =nElemId+(j+1)+NXElems+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems

                  nPos=0
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos] = int(nGNodeId1)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+1] = int(nGNodeId2)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+2] = int(nGNodeId3)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+3] = int(nGNodeId4)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+4] = int(nGNodeId5)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+5] = int(nGNodeId6)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+6] = int(nGNodeId7)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+7] = int(nGNodeId8)
                  #print('Elem_LocaltoGlobalNodeArr[nElemId]',Elem_LocaltoGlobalNodeArr[nElemId])
              
                  count+= 1
    if(order==2):
      for i in range(NZElems):#in z direction
          for j in range(NYElems):#in y direction
              for k in range(NXElems):#in x direction
                  #first looping through all X dir elems then to Y dir and then to Z dir
                  nElemId = k+(j)*NXElems+(i)*NXElems*NYElems

                  nGNodeId1 =nElemId+(j)+(i)*(NXElems+NYElems+1)
                  nGNodeId2 =nElemId+(j+1)+(i)*(NXElems+NYElems+1)
                  nGNodeId3 =nElemId+(j+1)+NXElems+1+(i)*(NXElems+NYElems+1)
                  nGNodeId4 =nElemId+(j+1)+NXElems+(i)*(NXElems+NYElems+1)
                  nGNodeId5 =nElemId+(j)+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                  nGNodeId6 =nElemId+(j+1)+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                  nGNodeId7 =nElemId+(j+1)+NXElems+1+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                  nGNodeId8 =nElemId+(j+1)+NXElems+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                  nGNodeId9 =nElemId  + (k)+ (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId10 =nElemId + (NXElems+k+1) + (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId11 =nElemId + (2*NXElems+1+k) + (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId12 =nElemId + (NXElems+k) + (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId13 =nElemId   + (k)+ (j+1)*(2*NXElems+1) + (i+1)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId14 =nElemId + (NXElems+k+2) + (j)*(2*NXElems+1) + (i+1)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId15 =nElemId + (2*NXElems+1+k+1) + (j)*(2*NXElems+1) + (i+1)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId16 =nElemId + (NXElems+k+1) + (j)*(2*NXElems+1) + (i+1)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId17 =nElemId  + ((NXElems+1)*NYElems + (NYElems+1)*NXElems + k+1)+ (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId18 =nElemId  + ((NXElems+1)*NYElems + (NYElems+1)*NXElems + k+2)+ (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId19 =nElemId  + ((NXElems+1)*NYElems + (NYElems+1)*NXElems + k +1+ NXElems+2)+ (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));
                  nGNodeId20 =nElemId  + ((NXElems+1)*NYElems + (NYElems+1)*NXElems + k +1+ NXElems+1)+ (j)*(2*NXElems+1) + (i)*((NXElems+1)*(2*NYElems+1)+NXElems*(NYElems+1));

                  nPos=0
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos] = int(nGNodeId1)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+1] = int(nGNodeId2)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+2] = int(nGNodeId3)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+3] = int(nGNodeId4)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+4] = int(nGNodeId5)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+5] = int(nGNodeId6)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+6] = int(nGNodeId7)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+7] = int(nGNodeId8)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+8] = int(nGNodeId1)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+9] = int(nGNodeId2)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+10] = int(nGNodeId3)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+11] = int(nGNodeId4)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+12] = int(nGNodeId5)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+13] = int(nGNodeId6)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+14] = int(nGNodeId7)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+15] = int(nGNodeId8)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+16] = int(nGNodeId1)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+17] = int(nGNodeId2)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+18] = int(nGNodeId3)
                  Elem_LocaltoGlobalNodeArr[nElemId][nPos+19] = int(nGNodeId4)
            
    return Elem_LocaltoGlobalNodeArr

def GetGNodeCordsArr( NXElems,  NYElems,  NZElems, NNodesPerElem,  dim,  xMin,  yMin,  zMin, xMax,  yMax,  zMax):
    xLength = xMax - xMin
    yLength = yMax - yMin
    zLength = zMax - zMin
    xLe = xLength/NXElems #length of each elem in x direction
    yLe = yLength/NYElems #length of each elem in y direction
    zLe = zLength/NZElems #length of each elem in z direction
    nNodesX = NXElems+1
    nNodesY = NYElems+1
    nNodesZ = NZElems+1

    Elem_GlobalNodeCordsArr = np.zeros(( (NXElems+1)*(NYElems+1)*(NZElems+1),dim) )

    count = 0

    for i in range(NZElems+1): # in z direction
        for j in range(NYElems+1):#in y direction
            for k in range(NXElems+1):#in x direction
                nodeID = k+(j)*(NXElems+1)+(i)*(NXElems+1)*(NYElems+1)
                Elem_GlobalNodeCordsArr[nodeID][0]=(k)*xLe; 
                Elem_GlobalNodeCordsArr[nodeID][1]=(j)*yLe; 
                Elem_GlobalNodeCordsArr[nodeID][2]=(i)*zLe;   

                count=+1
    return Elem_GlobalNodeCordsArr

# Elasticity tensor Cijkl = dS_IJ/dE_KL
def Cijkl( i,  j,   k, l):
    lambdaVal = 6*pow(10,10)
    Mu = 2*pow(10,10)
    return Mu * ((i==k)*(j==l) + (i==l)* (j==k)) + lambdaVal * (i==j) * (k==l)

def delta(i, k):
    return (i==k)

#K is Assembly of Geometric and Material stiffness matrix and 
#R is the matrix NdashJ*FiJ*SIJ
def GetKandR3DFE( NElemsX,  NElemsY,  NElemsZ, GNodeCordsArr, ElemArrQuad, displacementVec, order, boundaryCondition, F):
    #Get Inputs 
    NElemsX, NElemsY, NElemsZ, NumOfElems, NumNodesTotal, NPerElems, dim, xMin, yMin, zMin, xMax, yMax, zMax, order = GetInputElementNodeAndDim()
    NumNodesPerElem = NPerElems
    lambdaVal = 6*pow(10,10)
    Mu = 2*pow(10,10)


    quadPts = [-0.7746,0,0.7746]   #-sqrt(3/5),0,sqrt(3/5)  
    NumQuadPts = len(quadPts)
    WeightquadPts = [0.5555,0.8888,0.5555]  #5/9,8/9,5/9  

    KGlobal = np.zeros((dim*NumNodesTotal,dim*NumNodesTotal))
    ReGlobal = np.zeros((dim*NumNodesTotal,1))
    
    FGlobal = np.zeros((dim*NumNodesTotal,1))
    #print('NumElems',NumOfElems)
    for i in range(NumOfElems):
        Klocal = np.zeros( (NumNodesPerElem*dim,NumNodesPerElem*dim) )
        ReLocal = np.zeros( (NumNodesPerElem*dim,1) )
        Flocal = np.zeros( (NumNodesPerElem*dim,1) )
        elemNodeLs = [int(x) for x in ElemArrQuad[i]]

        for j in range(NumQuadPts):
            for k in range(NumQuadPts):
                for l in range(NumQuadPts):
                    #For Klocal
                    N = GetShapeFn(quadPts[j],quadPts[k],quadPts[l],order)
                    NwZ = GetShapeFnGradientwZ(quadPts[j],quadPts[k],quadPts[l],order)
                    NwEta = GetShapeFnGradientwEta(quadPts[j],quadPts[k],quadPts[l],order)
                    NwK = GetShapeFnGradientwK(quadPts[j],quadPts[k],quadPts[l],order)
                    #getting all nodes to the corresponding element
                    nodeIds = ElemArrQuad[:][0]
                    a0 = np.arange(1) #to pass 0 as arg to matrix slicing np.ix_(NodeIdls,a)
                    a1 = np.arange(1,2)
                    a2 = np.arange(2,3)
                    Jacobian = np.zeros( (dim,dim) )
                    # print('NwZ',NwZ.shape)
                    # print('NwZK',NwK.shape)
                    # print('NwEta',NwEta.shape)
                    # print('GNodeCordsArr',GNodeCordsArr.shape)
                    Jacobian = np.array((np.dot(NwZ,GNodeCordsArr[np.ix_(elemNodeLs, a0)]),   np.dot(NwEta,GNodeCordsArr[np.ix_(elemNodeLs, a0)]),  np.dot(NwK,GNodeCordsArr[np.ix_(elemNodeLs, a0)]), np.dot(NwZ,GNodeCordsArr[np.ix_(elemNodeLs, a1)]), np.dot(NwEta,GNodeCordsArr[np.ix_(elemNodeLs, a1)]), np.dot(NwK,GNodeCordsArr[np.ix_(elemNodeLs, a1)]), np.dot(NwZ,GNodeCordsArr[np.ix_(elemNodeLs, a2)]), np.dot(NwEta,GNodeCordsArr[np.ix_(elemNodeLs, a2)]), np.dot(NwK,GNodeCordsArr[np.ix_(elemNodeLs, a2)])))
                    Jacobian = np.reshape(Jacobian, (dim, dim))
                    invJacobian = np.linalg.inv(Jacobian)

                    NdashZEtaX = np.zeros( (NPerElems,dim) )
                    NdashZEtaX[:,0]= NwZ
                    NdashZEtaX[:,1]= NwEta
                    NdashZEtaX[:,2]= NwK
                    NdashX = np.matmul(NdashZEtaX, invJacobian) #Nfirst
                    dispVecForCurElemNodes = displacementVec[:,elemNodeLs]
                    GradU = np.matmul(dispVecForCurElemNodes,NdashX)

                    deformGradF = np.identity(dim) + GradU #F_IK
                    #Green lagrange strain E_IJ
                    GreenLagrangeE = 0.5*(np.matmul(np.transpose(deformGradF),deformGradF) - np.identity(dim))
                    traceE = 0 #E_KK
                    for e in range(dim):
                      traceE += GreenLagrangeE[e,e]
                    
                    #St. Venant Kirchoff energy density function
                    #SIJ = 2µEIJ + λδIJEKK , where λ = 6x1010 Pa, µ = 2x1010 Pa
                    S_IJ=np.zeros((dim,dim))
                    for E in range(dim):
                      for F in range (dim):
                        S_IJ[E,F] = 2* Mu * GreenLagrangeE[E,F] + lambdaVal * (E==F) * traceE
                    
                    #P = FS
                    #P_IJ = np.dot(deformGradF,S_IJ)

                    kL = np.zeros( (NumNodesPerElem*dim,NumNodesPerElem*dim) )
                    mL = np.zeros( (NumNodesPerElem*dim,NumNodesPerElem*dim) )

                    for m in range(NumNodesPerElem):
                        for n in range(NumNodesPerElem):
                          
                          #Geometric stiffness
                          for a in range(dim):
                            for b in range(dim):
                              for A in range(dim):
                                for B in range(dim):
                                  if(i==j):
                                    Klocal[dim*(m-1)+a, dim*(n-1)+b] += NdashX[n,A]*S_IJ[A,B]*NdashX[m,B]*abs(np.linalg.det(Jacobian))*WeightquadPts[j]*WeightquadPts[k]*WeightquadPts[l]
                            
                          #Material stiffness
                          for a in range(dim):
                            for b in range(dim):
                              for B in range(dim):
                                for L in range(dim):
                                  for A in range(dim):
                                    for K in range(dim):
                                      Klocal[dim*(m-1)+a, dim*(n-1)+b] += NdashX[m,B]*deformGradF[a,A]*Cijkl(A,B,K,L)*deformGradF[b,K]*NdashX[n,L]*abs(np.linalg.det(Jacobian))*WeightquadPts[j]*WeightquadPts[k]*WeightquadPts[l]
                    #Getting local R vector N,J*Fi,J*SI,J
                    for m in range(NumNodesPerElem):
                      for b in range(dim):
                        for A in range(dim):
                          for B in range(dim):
                            ReLocal[dim*m+b,0] += NdashX[m,B]*deformGradF[b,A]*S_IJ[A,B]*abs(np.linalg.det(Jacobian))*WeightquadPts[j]*WeightquadPts[k]*WeightquadPts[l]               

        #Assembling the KGlobal and ReGlobal
        for m in range(NumNodesPerElem):
          a1 = np.arange(3*m,3*(m+1))
          I=np.arange(3*(int(ElemArrQuad[i][m])),3*int(ElemArrQuad[i][m])+3)
          for n in range(NumNodesPerElem):
              J=np.arange(3*(int(ElemArrQuad[i][n])),3*int(ElemArrQuad[i][n])+3)
              b1 = np.arange(3*n,3*(n+1))
              KGlobal[np.ix_(I, J)]+= Klocal[np.ix_(a1, b1)]
          a0 = np.arange(1)
          ReGlobal[np.ix_(I, a0)]+= ReLocal[np.ix_(a1, a0)]
    return KGlobal, FGlobal, ReGlobal

def ApplyBoundaryConditions(GNodeCordsArr, NumNodesTotal, numloadSteps, nLoadStep, KGlobal, ReGlobal, dim, xMin, yMin, zMin, xMax, yMax, zMax,BC):

    #Applying Dirichilet Boundary conditions
    #BC1
    #   3D Hexahedral Mesh: ([0, 0.1]x[0, 0.03]x[0, 0.03], use a 10 x 3 x 3 element mesh.)
    #   u1 = 0 on x1 = 0, u1 = 1 on x1 = 10. Additionally, u2, u3 = 0 at  x = 0, and u2 =0 at x = 3e3.
    
    #BC2. Dirichlet boundary conditions: u = 0 on x1 = 0, u1 = 1 on x1 = 10.

    UatXMax = 0.01 #displacement at x=xMax
    g2=(UatXMax/numloadSteps)*nLoadStep
    UatX3e3 = 0 #at x = 3e3
    g1 = (UatX3e3/numloadSteps)*nLoadStep
    #Getting Dirchlet Nodes 
    if (BC==1): #first boundary condition
      #ApplyBC
      lstDirichBC = []            
      j =0
      for i in range(NumNodesTotal):
        if(GNodeCordsArr[i][0]==0):   #at x=0
            new=[]
            new.append(int(3*i))
            new.append(0)
            lstDirichBC.append(new)

            new=[]
            new.append(int(3*i+1))
            new.append(0)
            lstDirichBC.append(new)

            new=[]
            new.append(int(3*i+2))
            new.append(0)
            lstDirichBC.append(new)
            j=j+3
        if(GNodeCordsArr[i][0]==xMax):   #at x=xMax
            new=[]
            new.append(int(3*i))
            new.append(g2)
            lstDirichBC.append(new)
            j=j+1
        if(GNodeCordsArr[i][2]==zMax):   #at z=zMax
            new=[]
            new.append(int(3*i+1))
            new.append(g1)
            lstDirichBC.append(new)
            j=j+1
    elif(BC==2): #second boundary condition
      lstDirichBC = []            
      j =0
      for i in range(NumNodesTotal):
        if(GNodeCordsArr[i][0]==0):   #at x=0
            new=[]
            new.append(int(3*i))
            new.append(0)
            lstDirichBC.append(new)

            new=[]
            new.append(int(3*i+1))
            new.append(0)
            lstDirichBC.append(new)

            new=[]
            new.append(int(3*i+2))
            new.append(0)
            lstDirichBC.append(new)
            j=j+3
        if(GNodeCordsArr[i][0]==xMax):   #at x=xMax
            new=[]
            new.append(int(3*i))
            new.append(g2)
            lstDirichBC.append(new)
            j=j+1


    lstDirichBC = np.array(lstDirichBC)
    #Get unique Dirich nodes
    lst1 = [int(x) for x in np.unique(lstDirichBC[:,0])]
    DirichBC = []
    for i in lst1:
      idx = np.nonzero(lstDirichBC[:,0] == i)[0][0]
      DirichBC.append([i,lstDirichBC[idx,1]])
    
    DirichBC = np.array(DirichBC)
    nDirichletnodes=len(DirichBC[:,0])
    nNotDirchiletnodes = dim*NumNodesTotal-nDirichletnodes

    #KDGlobal, MDGlobal and FDGlobal Dirichlet
    KDGlobal = np.zeros((nNotDirchiletnodes,nNotDirchiletnodes))
    FDGlobal = np.zeros((nNotDirchiletnodes,1))
    NotDirchNodes = np.zeros((nNotDirchiletnodes,1))
    ReDGlobal = np.zeros((nNotDirchiletnodes,1))
  
    #Getting NonDirchlet Nodes 
    count=0
    for i in range(dim*NumNodesTotal):
      flag=0
      for k in range(nDirichletnodes):
          if(i==DirichBC[k][0]):
              flag=1
      if(flag==0):
          NotDirchNodes[count][0]=i
          count +=1

    #KDGlobal & ReDGlobal
    for k in range(nNotDirchiletnodes):
      for l in range(nNotDirchiletnodes):
        KDGlobal[k][l]=KGlobal[int(NotDirchNodes[k][0])][int(NotDirchNodes[l][0])]

    #fdash
    F_forSubstract = np.zeros((nNotDirchiletnodes,1))
    for i in range(nNotDirchiletnodes):
      for j in range(nDirichletnodes):
        F_forSubstract[i][0]=KGlobal[int(NotDirchNodes[i][0])][int(DirichBC[j][0])]*DirichBC[j][1] + F_forSubstract[i][0]

    #ReDGlobal
    for i in range(nNotDirchiletnodes):
      ReDGlobal[i][0]=ReGlobal[int(NotDirchNodes[i][0])][0] - F_forSubstract[i][0]
    return KDGlobal, ReDGlobal, DirichBC, NotDirchNodes


def GetDGlobalArr( NElemsX,  NElemsY,  NElemsZ,  order,  dim,  boundaryCondition, F, transient, xMin, yMin, zMin, xMax, yMax, zMax, numloadSteps):
    NumOfElems = NElemsX*NElemsY*NElemsZ
    NPerElems =8 #nodes per Elem

    NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1)

    if (order==1):
        NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1) #linear
        NPerElems = 8
    
    #createmesh
    ElemArrQuad = GetElemArr(NElemsX, NElemsY, NElemsZ, NPerElems, order)
    #Get Node coords
    GNodeCordsArr = GetGNodeCordsArr(NElemsX, NElemsY,NElemsZ, NPerElems ,dim, xMin, yMin, zMin, xMax, yMax, zMax)
    
    DGlobal_Arr = np.zeros((dim*NumNodesTotal,1)) #Initialze displacement vector array to append data after each loadstep
    DGlobalOld = np.zeros((dim*NumNodesTotal,1)) #Initialze displacement vector
    tol = 1.0e-6
    #Implement Newton Raphson's method
    for i in range(numloadSteps):
      nextIterationbool = True
      counterIter = 0
      while (nextIterationbool and counterIter < 10):
        print('TonextIter')

        counterIter+=1
        displacementVec = np.reshape(DGlobalOld, (dim, NumNodesTotal))

        #Getting KGlobal, MGlobal, FGlobal
        KGlobal, FGlobal, ReGlobal = GetKandR3DFE(NElemsX, NElemsY, NElemsZ, GNodeCordsArr, ElemArrQuad, displacementVec, order, boundaryCondition, F)
        print('ReGlobal',ReGlobal.shape)
      
        #Apply BoundaryC
        nLoadStep = i+1
        KDGlobal, ReDGlobal, DirichBC, NotDirchNodes = ApplyBoundaryConditions(GNodeCordsArr, NumNodesTotal, numloadSteps, nLoadStep, KGlobal, ReGlobal, dim, xMin, yMin, zMin, xMax, yMax, zMax,boundaryCondition)
        nDirichletnodes=len(DirichBC[:,0]) #number of Dirichilet nodes
        nNotDirchiletnodes = dim*NumNodesTotal-nDirichletnodes #number of Non Dirichilet nodes

        #Initializing dirichlet Matrices
        DGlobal_D = np.zeros((dim*NumNodesTotal-nDirichletnodes,1))
        DNew_D = np.zeros((dim*NumNodesTotal-nDirichletnodes,1))
        NotDirchNodes = [int(x) for x in NotDirchNodes]

        DNew = np.zeros((dim*NumNodesTotal,1))
        deltaDGlobal = np.zeros((dim*NumNodesTotal,1))
        
        DGlobal_D[:,0]=DGlobalOld[NotDirchNodes,0]
        DGlobal_D = np.array(DGlobal_D)

        #deltaD = KD^-1 * ReD
        deltaDGlobal = np.matmul(np.linalg.inv(KDGlobal), ReDGlobal) 
       
        DNew_D = DGlobal_D + deltaDGlobal  #getting value of newD value from oldD and deltaD

        for m in range(nDirichletnodes):
            DNew[int(DirichBC[m][0])][0]=DirichBC[m][1]

        for n in range(nNotDirchiletnodes):
            DNew[int(NotDirchNodes[n])]=DNew_D[n][0]
            
        normDGlobalOld = np.linalg.norm(DGlobalOld)
        normDNew = np.linalg.norm(DNew)
        print('normDGlobalOld',normDGlobalOld)
        print('normDNew',normDNew)
        maxNorm=max(normDNew,normDGlobalOld)
        print('normDNew/maxNorm',normDNew/maxNorm)

        DNorm = DGlobalOld - DNew
        normDNorm = np.linalg.norm(DNorm)
        print('normDNorm',normDNorm)
        if (normDNorm < tol):
          nextIterationbool = False
        
        # #assigning values from new D vector to DGlobalOld
        DGlobalOld = DNew
      print('DNew',DNew)
      DGlobal_Arr = np.append(DGlobal_Arr, DNew, 1)
      print('DGlobal_Arr',DGlobal_Arr)

    for i in range(numloadSteps):
      i+=1
      filename = 'HW4DGlobalArr4440Elems'+str(i)+'.txt'
      np.savetxt(filename, DGlobal_Arr[:,i], fmt='%1.7f')
    print("DGlobalDone\n")
    return ElemArrQuad, GNodeCordsArr, DGlobal_Arr


def GetInputElementNodeAndDim():
    NElemsX =10 #X 
    NElemsY =3 # Y 
    NElemsZ =3 # Z direction

    NElemsX =1 #horizontal
    NElemsY =1 # vertical
    NElemsZ =1  #/ Z direction
    NumOfElems = NElemsX*NElemsY*NElemsZ
    NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1)
    NPerElems =8 #nodes per Elem
    dim = 3 #DOF
    #Geometry
    xMin = 0.0
    yMin = 0.0
    zMin = 0.0
    xMax = 0.1
    yMax = 0.03
    zMax = 0.03

    order = 1 #1-linear, 2-Quadrilateral
    if (order==1):
      NPerElems =8 #nodes per Elem
      NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1)
    elif(order==2):
      NPerElems =20 #nodes per Elem for 2nd order element
      NumNodesTotal = (2*NElemsX+1)*(2*NElemsY+1)*(2*NElemsZ+1)

    return  NElemsX, NElemsY, NElemsZ, NumOfElems, NumNodesTotal, NPerElems, dim, xMin, yMin, zMin, xMax, yMax, zMax, order

def main():
    #Get Inputs
    NElemsX, NElemsY, NElemsZ, NumOfElems, NumNodesTotal, NPerElems, dim, xMin, yMin, zMin, xMax, yMax, zMax, order = GetInputElementNodeAndDim()
    transient=0 #0-Static 1-Dynamic

    F=0.0
    boundaryCondition = 1 #1 for BC1 and 2 for BC2
    numloadSteps = 1

    #Getting DGlobal
    print("DGlobal Start\n")
    ElemArrQuad, GNodeCordsArr, DGlobalArr = GetDGlobalArr(NElemsX, NElemsY,NElemsZ, order, dim, boundaryCondition, F, transient, xMin, yMin, zMin, xMax, yMax, zMax, numloadSteps)
    np.savetxt('HW4DGlobalArrmain.txt', DGlobalArr, fmt='%1.7f')
    print("DGlobal Done\n")
    print("write data to the file\n")

    #Write mesh array for output
    ElemArrQuadForPrint = np.zeros( (NElemsX*NElemsY*NElemsZ,NPerElems) )
    for i in range(NumOfElems):
        for x in range(NPerElems):
          ElemArrQuadForPrint[i][x] = ElemArrQuad[i][x]+1
    np.savetxt('HW4ElemArr4440Elems.txt', ElemArrQuadForPrint,fmt='%i')
    
    #Write node coordinates array 
    np.savetxt('HW4NodeCoordsArr4440Elems.txt', GNodeCordsArr,fmt='%1.2f')
    print("Completed writing data to the file\n")
    
    return 0

main()


DGlobal Start

TonextIter
ReGlobal (24, 1)
normDGlobalOld 0.0
normDNew 0.020517528155570768
normDNew/maxNorm 1.0
normDNorm 0.020517528155570768
TonextIter
ReGlobal (24, 1)
normDGlobalOld 0.020517528155570768
normDNew 0.04602213428171956
normDNew/maxNorm 1.0
normDNorm 0.04075062017265786
TonextIter
ReGlobal (24, 1)
normDGlobalOld 0.04602213428171956
normDNew 0.08580468548106543
normDNew/maxNorm 1.0
normDNorm 0.050854933120261354
TonextIter
ReGlobal (24, 1)
normDGlobalOld 0.08580468548106543
normDNew 0.43467128997571497
normDNew/maxNorm 1.0
normDNorm 0.3650484813252372
TonextIter
ReGlobal (24, 1)
normDGlobalOld 0.43467128997571497
normDNew 2.6978009158572784
normDNew/maxNorm 1.0
normDNorm 2.3359960682267804
TonextIter
ReGlobal (24, 1)
normDGlobalOld 2.6978009158572784
normDNew 16.250268126679217
normDNew/maxNorm 1.0
normDNorm 13.732491717775913
TonextIter
ReGlobal (24, 1)
normDGlobalOld 16.250268126679217
normDNew 104.16780185510511
normDNew/maxNorm 1.0
normDNorm 87.94835085110458
Tonext

0

In [ ]:
xMin = 0.0
yMin = 0.0
zMin = 0.0
xMax = 0.1
yMax = 0.03
zMax = 0.03
NElemsX =10 #horizontal
NElemsY =3 # vertical
NElemsZ =3 # Z direction
UatXMax = 0.01 #displacement at x=xMax
numloadSteps = 10
g2=UatXMax/numloadSteps
UatX3e3 = 0 #at x = 3e3
g1 = UatX3e3/numloadSteps
NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1)
order=1
if (order==1):
    NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1) #linear
    NPerElems = 8
dim=3
#createmesh
#ElemArrQuad = GetElemArr(NElemsX, NElemsY, NElemsZ, NPerElems)
#Get Node coords
GNodeCordsArr = GetGNodeCordsArr(NElemsX, NElemsY,NElemsZ, NPerElems ,dim, xMin, yMin, zMin, xMax, yMax, zMax)

DirichBC = []            
j =0
for i in range(NumNodesTotal):
    if(GNodeCordsArr[i][0]==0):   #at x=0
        new=[]
        new.append(int(3*i))
        new.append(0)
        DirichBC.append(new)

        new=[]
        new.append(int(3*i+1))
        new.append(0)
        DirichBC.append(new)

        new=[]
        new.append(int(3*i+2))
        new.append(0)
        DirichBC.append(new)
        j=j+3
    if(GNodeCordsArr[i][0]==xMax):   #at x=xMax
        new=[]
        new.append(int(3*i))
        new.append(g2)
        DirichBC.append(new)
        j=j+1
    if(GNodeCordsArr[i][2]==zMax):   #at x=xMax
        new=[]
        new.append(int(3*i+1))
        new.append(g1)
        DirichBC.append(new)
        j=j+1

DirichBC = np.array(DirichBC)
nDirichletnodes=len(DirichBC[:,0])
nNotDirchiletnodes = dim*NumNodesTotal-nDirichletnodes


In [ ]:
DGlobalOld = [0,1,0]
DGlobalOld = np.asarray(DGlobalOld)
DNew = [1,0,0]
DNew = np.asarray(DNew)

normDGlobal = np.linalg.norm(DGlobalOld)
normDNew = np.linalg.norm(DNew)
print(abs(normDGlobal-normDNew))

DNorm = DGlobalOld - DNew
normDNew = np.linalg.norm(DNorm)
print(DNorm)
print(normDNew)


0.0
[-1  1  0]
1.4142135623730951


In [ ]:
list1 = [1, 5, 7]
new1 = [a[:,list1] for i in list1]

a[np.ix_(0, 1)]

In [ ]:
a[:,list1]

In [ ]:
NdashX  = [[-7.8730129  -7.8730129  -0.78730129],
 [ 7.8730129  -0.9999871  -0.09999871],
 [ 0.9999871   0.9999871  -0.01270129],
 [-0.9999871   7.8730129  -0.09999871],
 [-0.9999871  -0.9999871   0.78730129],
 [ 0.9999871  -0.1270129   0.09999871],
 [ 0.1270129   0.1270129   0.01270129],
 [-0.1270129   0.9999871   0.09999871]]

SyntaxError: ignored

In [ ]:
#Newton Raphson method
import numpy as np
def func(x):
  return pow(x,3)+2*x-2

def funcDerivative(x):
  return 3*pow(x,2)+2

def newtonRaphson(x):
  xn_1 = x
  fval = func(xn_1)
  deffval = funcDerivative(xn_1)
  xn = xn_1 - fval/deffval
  return xn

def main():

  intialGuess = 0
  x = intialGuess
  xn = newtonRaphson(intialGuess)
  while (abs(x - xn) > 1.0e-6):
    x=xn
    print(xn)
    xn = newtonRaphson(x)
  print(xn)

main()

1.0
0.8
0.7714285714285715
0.7709171570288672
0.7709169970592638


In [ ]:
import numpy as np
A = [1,2,4,6]
Arr = [[7,8,9,10,11,12,13],[17,18,19,110,111,112,113]]
Arr=np.asarray(Arr)
#Arr[NotDirchNodes]=DGlobal_D[:,i]
#print(Arr[0,:])
if (2 not in Arr[0,:]):
  print('Arr',Arr[0,:] )

Arr [ 7  8  9 10 11 12 13]


In [ ]:
numbers=[1,2,3,4]
print(numbers[2::])

[3, 4]


In [ ]:
DirichBC = [[0, 0], [1, 0], [2, 0]]
DirichBC=np.asarray(DirichBC)
DirichBC = np.append(DirichBC, [[1,2]], axis=0)
DirichBC

array([[0, 0],
       [1, 0],
       [2, 0],
       [1, 2]])

In [ ]:
DirichBC[:,0]

array([0, 1, 2, 1])

In [ ]:
pow(2,2)

4